## LEKOUNDA NGOLO Mardochet Gedeon
## COOP MSC | DATA SCIENCE
### ASSIGNMENT 3 :  DATA COLLECTION
#### 23-11-2025

### <font color = 'gold'> Import packages


In [1]:
# import packages
from requests import get
from bs4 import BeautifulSoup as bs
import pandas as pd

### <font color = 'gold'> Collect data over one page

First, we will start with one container and scrape the data. In that container, we will first try to get the URL of the container and enter inside the container to scrape the informations needed. After we generalize the scraping to the other containers by using a for loop.

### <font color = 'gold'> Collect data over multiple pages

In [ ]:
# create a empty dataframe df

# loop over pages indexes


### <font color = 'gold'> Assignment

In [2]:
# url
url = 'https://books.toscrape.com/catalogue/page-2.html'
# scrape over all pages the following informations : title, price , avalaibility, number of product, description, Product Type
# Clean the data

In [3]:
from requests import get
from bs4 import BeautifulSoup as bs
import pandas as pd

# I create a function for page scrapping
def scrape_page(page_number):
    url = f'https://books.toscrape.com/catalogue/page-{page_number}.html'
    res = get(url)

    # stop if pages doesn't exist
    """if res.status_code != 200:
        return []"""
    soup = bs(res.content, "html.parser")
    containers = soup.find_all("article", class_="product_pod")
    data = []
    for container in containers:
        product_rel_link = container.find('a')['href']
        product_url ='https://books.toscrape.com/catalogue/' + product_rel_link
        sub_container = get(product_url)
        soup_container =bs(sub_container.content, 'html.parser')
        try:
            # Handle title of page
            title = soup_container.find('div', 'col-sm-6 product_main').h1.text
            # handle price of the book
            price = soup_container.find('p', 'price_color').text.replace('£', '')
            # handle the avalaibility
            gen_inf = soup_container.find('p', 'instock availability').text.split()
            avalaibility =" ".join(gen_inf[:2])
            number_prod = gen_inf[2].replace('(','')
            # handle description
            header = soup_container.find("div", id="product_description")
            description = None
            if header:
                p = header.find_next("p")
                if p:
                    description = p.get_text(strip=True)
            product_type = soup_container.find("th", string="Product Type").find_next("td").text.strip()
             # Dictionnary for datable
            info_dict = {}
            info_table = soup_container.find("table", class_="table table-striped")
            if info_table:
                rows = info_table.find_all("tr")
                for row in rows:
                    th = row.find("th")
                    td = row.find("td")
                    if th and td:
                        key = th.text.strip()
                        value = td.text.strip().replace("£", "")
                        info_dict[key] = value
            # Extract data table values
            upc = info_dict.get("UPC")
            product_type = info_dict.get("Product Type")
            price_excl_tax = info_dict.get("Price (excl. tax)")
            price_incl_tax = info_dict.get("Price (incl. tax)")
            tax = info_dict.get("Tax")
            availability_full = info_dict.get("Availability")
            number_reviews = info_dict.get("Number of reviews")
            # add into dictionnary
            data.append({
              'title': title,
              #'price': price, # same as price_excl_tax
              'avalaibility': avalaibility,
              'number_product': number_prod,
              'description': description,
              'tax': tax,
              'number_reviews': number_reviews,
              'upc': upc,
              'price_excl_tax': price_excl_tax,
              'price_incl_tax': price_incl_tax,
              'availability_full': availability_full,
              'product_type': product_type,
              'url': product_url
            })
        except Exception as e:
            # on ignore l’erreur mais continue proprement
            continue
    return data

In [4]:
df = pd.DataFrame()
for page in range(1, 51):
    print(f"Scraping page {page}...")
    page_data = scrape_page(page)
    if len(page_data) == 0:
        print("Probably empty page.")
        break
    df_page = pd.DataFrame(page_data)
    df = pd.concat([df, df_page], axis=0).reset_index(drop=True)
print("Scraping completed!") # For test need I scrapping only 4 pages

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Scraping page 21...
Scraping page 22...
Scraping page 23...
Scraping page 24...
Scraping page 25...
Scraping page 26...
Scraping page 27...
Scraping page 28...
Scraping page 29...
Scraping page 30...
Scraping page 31...
Scraping page 32...
Scraping page 33...
Scraping page 34...
Scraping page 35...
Scraping page 36...
Scraping page 37...
Scraping page 38...
Scraping page 39...
Scraping page 40...
Scraping page 41...
Scraping page 42...
Scraping page 43...
Scraping page 44...
Scraping page 45...
Scraping page 46...
Scraping page 47...
Scraping page 48...
Scraping page 49...
Scraping page 50...
Scraping 

In [6]:
df["price_excl_tax"] = df["price_excl_tax"].astype(float)

In [7]:
df["product_type"] = df["product_type"].str.replace("\n", " ").str.strip()
df["description"] = df["description"].str.replace("\n", " ").str.strip()

In [8]:
df.head()

,title,avalaibility,number_product,description,tax,number_reviews,upc,price_excl_tax,price_incl_tax,availability_full,product_type,url
0,A Light in the Attic,In stock,22,It's hard to imagine a world without A Light i...,0.00,0,a897fe39b1053632,51.77,51.77,In stock (22 available),Books,https://books.toscrape.com/catalogue/a-light-i...
1,Tipping the Velvet,In stock,20,"""Erotic and absorbing...Written with starling ...",0.00,0,90fa61229261140a,53.74,53.74,In stock (20 available),Books,https://books.toscrape.com/catalogue/tipping-t...
2,Soumission,In stock,20,"Dans une France assez proche de la nôtre, un h...",0.00,0,6957f44c3847a760,50.10,50.10,In stock (20 available),Books,https://books.toscrape.com/catalogue/soumissio...
3,Sharp Objects,In stock,20,"WICKED above her hipbone, GIRL across her hear...",0.00,0,e00eb4fd7b871a48,47.82,47.82,In stock (20 available),Books,https://books.toscrape.com/catalogue/sharp-obj...
4,Sapiens: A Brief History of Humankind,In stock,20,From a renowned historian comes a groundbreaki...,0.00,0,4165285e1663650f,54.23,54.23,In stock (20 available),Books,https://books.toscrape.com/catalogue/sapiens-a...


In [9]:
df.shape

(1000, 12)

In [10]:
print(df.duplicated().sum())

0
